In [1]:
import cv2
import sys
import math
import numpy as np
import os
import pysift

In [2]:
feature_object = cv2.ORB_create(50000)

In [3]:
FLANN_INDEX_LSH = 6
index_params = dict(algorithm = FLANN_INDEX_LSH,
                   table_number = 6,
                   key_size = 12,
                   multi_probe_level = 1)


In [4]:
search_params = dict(check=50)
matcher = cv2.FlannBasedMatcher(index_params,search_params)


#Finding Features in the first image

In [8]:
test_img_dir = '/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/LukaAllBoard2.jpg'

In [9]:

img = cv2.imread(test_img_dir,0)


In [10]:

keypoints, descriptors = feature_object.detectAndCompute(img,None)

#Detecting features in second frame

In [313]:
test_img2_dir = '/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/PZ.png'
img2 = cv2.imread(test_img2_dir,0)


In [314]:
keypoints_img2 , descriptors_img2 = feature_object.detectAndCompute(img2, None)


#Finding Matches

In [315]:
#matcher = cv2.BFMatcher()
matches = matcher.knnMatch(descriptors , trainDescriptors = descriptors_img2, k = 2)
good_matches = []
try:
   for (m,n) in matches:
       if m.distance < 0.7*n.distance:
           good_matches.append(m)
except ValueError:
   print("caught error - no matches from current frame")

#Drawing matches

In [316]:
source_pts = np.float32([keypoints[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
destination_pts = np.float32([ keypoints_img2[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)


In [317]:
draw_params = dict(matchColor= (0,255,0) , singlePointColor = (25,0,0) , flags= 0)
display_matches = cv2.drawMatches(img, keypoints , img2 , keypoints_img2, good_matches, None, **draw_params)

In [318]:
cv2.imwrite( '/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/PZBoardOrbMatches.png', 
           display_matches) 

True

#Demo

In [ ]:
videoPath ='/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/testVideo.mp4'
cap  = cv2.VideoCapture(videoPath)
while(cap.isOpened()):
    ret, img2 = cap.read()
    img2 = cv2.resize(img2 , (img.shape[1], img.shape[0]))
    if ret:
        keypoints_img2 , descriptors_img2 = feature_object.detectAndCompute(img2, None)
        
        matches = matcher.knnMatch(descriptors , trainDescriptors = descriptors_img2, k = 2)
        good_matches = []
        try:
           for (m,n) in matches:
               if m.distance < 0.7*n.distance:
                   good_matches.append(m)
        except ValueError:
           print("caught error - no matches from current frame")
        
        source_pts = np.float32([keypoints[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        destination_pts = np.float32([ keypoints_img2[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
        
        draw_params = dict(matchColor= (0,255,0) , singlePointColor = (25,0,0) , flags= 0)
        display_matches = cv2.drawMatches(img, keypoints , img2 , keypoints_img2, good_matches, None, **draw_params)
        
        cv2.imshow("Demo", display_matches)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        cap.release()
        break
cap.release()
cv2.destroyAllWindows()   
        

# Finding SIFT features

In [419]:
img = cv2.imread('/Users/sandeep/Desktop/FinalYearProject/YoutubeScreenshotScrapper/data/test/segmentedRimCopy.jpg', 0)

In [420]:
keypoints, descriptors = pysift.computeKeypointsAndDescriptors(img)

In [417]:
img2 = cv2.resize(img2,(250,250))
keypoints_img2 , descriptors_img2 = pysift.computeKeypointsAndDescriptors(img2)

In [421]:
# Initialize and use FLANN
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
matcher = cv2.FlannBasedMatcher(index_params, search_params)
matches = matcher.knnMatch(descriptors, descriptors_img2, k=2)